In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API 
from tweepy import Cursor

In [2]:
consumer_key='qcCU8SXNbYan2cAZ4apDqXP5r'
consumer_secret='kzUFaP9UHeK7yBofQZ1veZZP2Tf0fLX05UBWircs9wizJv95Jg'
access_token='1090283141414043649-O6xtwiJfCfKYWGLRXxMw9FolevIiN3'
access_secret='9WE1BYqyTvCqMkBb8WdpjtN4to4hWc4UL83wOg3ug3Hod'

In [3]:
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user
        
    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets

In [4]:
# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token,access_secret)
        return auth

In [5]:
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator() 

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)

In [6]:
# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)

In [8]:
if __name__ == '__main__':
 
    # Authenticate using config.py and connect to Twitter Streaming API.
    hash_tag_list = ["midas","iiitd"]
    fetched_tweets_filename = "tweets.json"

    twitter_client = TwitterClient('pycon')
    print(twitter_client.get_user_timeline_tweets(1))

[Status(_api=<tweepy.api.API object at 0x00000160283DCF60>, _json={'created_at': 'Fri Apr 05 23:13:00 +0000 2019', 'id': 1114304963759542272, 'id_str': '1114304963759542272', 'text': "Don't forget to visit the #pycon2019 Job Fair on Sunday, May 5! You won't find a better place to talk to companies… https://t.co/a97J0v4U5N", 'truncated': True, 'entities': {'hashtags': [{'text': 'pycon2019', 'indices': [26, 36]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/a97J0v4U5N', 'expanded_url': 'https://twitter.com/i/web/status/1114304963759542272', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [116, 139]}]}, 'source': '<a href="https://sproutsocial.com" rel="nofollow">Sprout Social</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 9475182, 'id_str': '9475182', 'name': 'PyCon', 'screen_name': 'pycon', 'location': 'Cleveland, OH', 'de

In [9]:
import numpy as np
import pandas as pd

In [36]:
class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

        
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['NumImages'] = np.array([len(tweet.entities['urls']) for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
        
        return df

In [37]:
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="midasIIITD", count=20)

    #print(dir(tweets[0]))
    #print(tweets[0].retweet_count)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    
    print(df.head(10))

                                              Tweets                date  \
0  RT @kdnuggets: Top 8 #Free Must-Read #Books on... 2019-04-06 17:11:29   
1  @nupur_baghel @PennDATS Congratulation @nupur_... 2019-04-06 16:43:27   
2  We have emailed the task details to all candid... 2019-04-05 16:08:37   
3  RT @rfpvjr: Our NAACL paper on polarization in... 2019-04-05 04:05:11   
4  RT @kdnuggets: Effective Transfer Learning For... 2019-04-05 04:04:43   
5  RT @stanfordnlp: What’s new in @Stanford CS224... 2019-04-03 18:31:53   
6  RT @DeepMindAI: Today we're releasing a large-... 2019-04-03 17:04:32   
7  RT @ylecun: Congratulations Jitendra Malik !\n... 2019-04-03 09:03:40   
8  RT @IIITDelhi: Another chance to take admissio... 2019-04-03 07:46:02   
9  Dear @midasIIITD internship candidates who hav... 2019-04-02 04:20:13   

   NumImages  likes  retweets  
0          1      0         2  
1          1     10         3  
2          1      8         1  
3          1      0        16  
4  

In [38]:
df['NumImages'] = df['NumImages'].where(df['NumImages']!=0, None)

In [39]:
df.head(10)

,Tweets,date,NumImages,likes,retweets
0,RT @kdnuggets: Top 8 #Free Must-Read #Books on...,2019-04-06 17:11:29,1,0,2
1,@nupur_baghel @PennDATS Congratulation @nupur_...,2019-04-06 16:43:27,1,10,3
2,We have emailed the task details to all candid...,2019-04-05 16:08:37,1,8,1
3,RT @rfpvjr: Our NAACL paper on polarization in...,2019-04-05 04:05:11,1,0,16
4,RT @kdnuggets: Effective Transfer Learning For...,2019-04-05 04:04:43,1,0,10
5,RT @stanfordnlp: What’s new in @Stanford CS224...,2019-04-03 18:31:53,None,0,57
6,RT @DeepMindAI: Today we're releasing a large-...,2019-04-03 17:04:32,None,0,844
7,RT @ylecun: Congratulations Jitendra Malik !\n...,2019-04-03 09:03:40,1,0,16
8,RT @IIITDelhi: Another chance to take admissio...,2019-04-03 07:46:02,None,0,4
9,Dear @midasIIITD internship candidates who hav...,2019-04-02 04:20:13,1,8,1


In [40]:
df['NumImages'].unique()

array([1, None, 2], dtype=object)